# **SmartPrep AI Agent**

In [1]:
!pip install -q transformers langchain faiss-cpu sentence-transformers accelerate huggingface_hub

In [2]:
!pip install -U langchain-community

In [3]:
!pip install -q PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.docstore.document import Document
from PyPDF2 import PdfReader
import torch

In [4]:
# Step 1: Load the LLaMA model
model_id = "meta-llama/Llama-3.2-1B"
print("⏳ Loading LLaMA model")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,dtype=torch.float16,device_map="auto")

pipe = pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=600,temperature=0.1,top_p=0.95)
llm = HuggingFacePipeline(pipeline=pipe)
print(" Model ready!\n")

⏳ Loading LLaMA model


Device set to use cuda:0


 Model ready!



/tmp/ipython-input-2184051400.py:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [5]:
# Step 2: Load PDF and convert to Documents

pdf_path = "/content/drive/MyDrive/SmartPrep job.pdf"
reader = PdfReader(pdf_path)

docs = []
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        docs.append(Document(page_content=text.strip(), metadata={"page_number": i+1}))

print(f"Loaded {len(docs)} job roles from PDF (each page = one role).")


Loaded 20 job roles from PDF (each page = one role).


In [6]:
# Step 3: Create embeddings + FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

/tmp/ipython-input-173327483.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
# Step 4: Define dynamic prompt function using PDF context

def dynamic_interview_prep(role, retrieved_docs):
    context_text = "\n".join([d.page_content for d in retrieved_docs])
    prompt = (
        f"You are an expert interview mentor.\n"
        f"Based on the following context:\n{context_text}\n\n"
        f"Generate 5 interview questions with short answers for the {role} position.\n"
        f"Also, list 3 important topics the candidate should revise."
    )
    return llm(prompt)

In [8]:
# Step 5: Console interface

print("🤖 SmartPrep AI Agent ready! (Type 'exit' or 'quit' to stop)\n")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("SmartPrep AI Agent: Goodbye and good luck! 🚀")
        break
    try:
        relevant_docs = retriever.get_relevant_documents(query)
        response = dynamic_interview_prep(query, relevant_docs)
        print("\nSmartPrep AI Agent:\n", response)
        print("\n📚 Retrieved Context Page(s):")
        for d in relevant_docs:
            print(f"- Page {d.metadata['page_number']}")
        print("\n" + "-"*80 + "\n")
    except Exception as e:
        print("⚠️ Error:", e)

🤖 SmartPrep AI Agent ready! (Type 'exit' or 'quit' to stop)

You: Python Developer


/tmp/ipython-input-204846254.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)
/tmp/ipython-input-477376143.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



SmartPrep AI Agent:
 You are an expert interview mentor.
Based on the following context:
1. Python Developer  
Key Skills:  Python, OOP, Pandas, NumPy, Flask, Django  
Core Concepts:  Data structures, algorithms, OOP, libraries  
Interview Questions:  
1. Explain Python’s GIL.  
2. How do you handle exceptions in Python?  
3. What are Python decorators?  
Topics to Revise:  OOP, Pandas, Flask

Generate 5 interview questions with short answers for the Python Developer position.
Also, list 3 important topics the candidate should revise.

📚 Retrieved Context Page(s):
- Page 1

--------------------------------------------------------------------------------

You: quit
SmartPrep AI Agent: Goodbye and good luck! 🚀
